# LLM ?
* LLM = Large Language Model，大型語言模型，用來處理和生成自然語言。
* 架構：基於 [Transformer](https://medium.com/@glowing_sage_deer_60/%E6%9D%8E%E5%AE%8F%E6%AF%85%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%922021-transformer-533006e3474) 生成。
* 輸入：文字（Token）；輸出：預測下一個 Token，直到結束。

## 生成式AI概念
在生成式AI開始流行前，受限於硬體與資料量，人們大部分只能使用有限的資源，用監督式的方式進行訓練(CNN、LSTM...)，而強化式學習的方法(GAN)只可能出現在論文的研究上，比較少看到實務上使用。大概2023年末開始，Google 團隊 OpenAI 開始起步線上開放給大家使用 chatGPT 後，大家開始發現強化式學習的厲害之處。

生成式AI訓練的方式，簡單用小偷作假鈔的例子當作範例:

<img src=".\img\img1.png" alt="drawing" width="800"/>

## 主流模型有哪些 (~2025)
| 模型 | 類型 | 開源 | 訓練參數量 | 運作方式 | 備註 |
| -------- | -------- | -------- |-------- | -------- | -------- |
| GPT-4     | API     | 否     |未公開     | 雲端推論     | 支援工具使用 |
| Claude 3     | API     | 否     |未公開     | 雲端推論     | 可處理大量上下文 |
| Gemini     | API     | 否     |未公開     | 雲端推論     | 整合 Google 生態|
| LLaMA 3    |開源     | 是     |8B / 70B     | 本地 / 雲皆可    | 授權較嚴格 |
| Mistral     | 開源     | 是     |7B     | 支援 GGUF     | 效能優、體積小 |
| Phi-2     | 開源    | 是     |2.7B     | 適合邊緣部署     | 微軟研究模型|
| Gemma    | 開源    | 是     |	2B / 7B     | 易於部署     | Google 出品 |

## 要建立自己的LLM，需要哪些東西
1. 確認目標
* 沒想寫code，只想用線上版GPT : OpenAI API / Claude / Gemini (補充:[OpenAI微調範例](https://www.accucrazy.com/openai-fine-tuning/))
* 想「部署模型」：選開源模型，考慮硬體資源
* 想「訓練模型」：需要數百萬以上資料 + GPU 叢集
2. 本地部署選項
就是已經有了模型，只需要安裝幾個軟體即可
* 工具選擇：llama.cpp（C++）、Ollama（簡單一行部署）、LM Studio（GUI）
* 支援格式：GGUF（用於量化後模型）
3. 推論條件
* 至少需：16GB RAM + 6~8GB VRAM（Mistral/Gemma/phi 可行）
* 高效部署工具：[vLLM](https://github.com/vllm-project/vllm)（商用級推論）、[Text Generation Inference](https://huggingface.co/docs/text-generation-inference/index)（HuggingFace）
4. 寫程式碼呼叫API
* 所需資訊：API Key、安全性管理、輸入輸出格式（JSON）
* 推薦套件：openai, anthropic, google.generativeai


# unsloth
github : https://github.com/unslothai/unsloth?tab=readme-ov-file
* 速成的LLM，特別針對訓練速度和低資源環境進行了優化
* 有寫好的很多開源模型做 Fine-tune 的程式碼，已經放在 Google Colab 上可以直接使用
* 僅支援單卡運算


# 基本環境準備
1. Python 環境（建議 Python 3.10 或以上）


In [ ]:
# python -m venv llm_env
# source llm_env/bin/activate

2. CUDA & 驅動
確保你的機器有 NVIDIA GPU，並安裝對應版本的：
* CUDA：11.8 或 12.x（視你 GPU 支援而定）
* cuDNN：與 CUDA 相符
* [安裝教學](https://medium.com/ching-i/win10-安裝-cuda-cudnn-教學-c617b3b76deb)

In [1]:
!nvidia-smi  

Sun May 25 17:23:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   44C    P0             13W /   90W |       0MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 安裝套件


In [ ]:
# pip install --upgrade pip
#看cuda版本
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu127 
# pip install unsloth
# pip install transformers accelerate peft bitsandbytes datasets


# 下載模型
Unsloth 支援的模型格式為 HuggingFace 格式，例如：

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,        # 自動選擇 torch.float16 or bfloat16
    load_in_4bit = True  # 使用 bitsandbytes 量化模型
)

# 範例快速部署（聊天測試）

In [ ]:
while True:
    prompt = input("You: ")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# 使用範例資料集:
[AWeirdDev/zh-tw-pts-articles-sm](https://huggingface.co/datasets/AWeirdDev/zh-tw-pts-articles-sm)->公視新聞網（news.pts.org.tw）所抓取的新聞資料。資料集共有 1,400 筆資料，每筆資料包含以下欄位：

    - image：縮圖
    - title：標題
    - conclusion：結論（「結論先講」一欄，可能為 None）
    - content：文章內容
    - timestamp：發布時間
    - category：類別（可能為 None）
    - link：文章連結

In [ ]:
# 快速確認格式：
from datasets import load_dataset

dataset = load_dataset("AWeirdDev/zh-tw-pts-articles-sm")
print(dataset)
print(dataset["train"][0])


#  Unsloth 與資料集怎麼搭配用？
假設你使用 LLaMA 3 8B 4bit 模型（Unsloth 最常見選項），並以 LoRA 做微調訓練：

In [ ]:
# 1. 載入模型
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

你可以根據任務設計 prompt，例如：

Chat 模型（問答導向）：

`Human: 以下是一篇文章，請幫我寫摘要：{text} \nAssistant:`

Summarization 模型（摘要導向）：

`請針對以下內文產生摘要：{text} \n摘要：`

In [ ]:
# 2. 準備資料（轉成 SFT 格式），需要轉為-> {"prompt": "...", "completion": "..."}
from datasets import load_dataset

raw_dataset = load_dataset("AWeirdDev/zh-tw-pts-articles-sm")

def convert(example):
    prompt = f"請閱讀以下內文，並撰寫摘要：\n{example['text']}\n\n摘要："
    completion = example['summary']
    return {"prompt": prompt, "completion": completion}

dataset = raw_dataset["train"].map(convert)



In [ ]:
# 3. 套用 tokenizer 與訓練
from unsloth import SFTTrainer
from transformers import TrainingArguments

# 使用 tokenizer 處理
dataset = dataset.map(lambda x: tokenizer(x["prompt"], x["completion"]), remove_columns=dataset.column_names)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = True,
        output_dir = "./lora-output",
        save_total_limit = 2,
    ),
)

trainer.train()


* 訓練完成後，你可以使用 model.save_pretrained() 或 peft_model.save_pretrained() 把 LoRA 存起來
* 輸出路徑： ./lora-output/  # 內有 adapter_config.json, adapter_model.bin 等


In [ ]:
# 一、載入微調後的模型
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from peft import PeftModel

# 載入原始模型（base）
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

# 套用微調的 LoRA 權重
model = PeftModel.from_pretrained(model, "./lora-output", is_trainable=False)
model.eval()  # 切成 eval 模式，避免 dropout 等訓練機制影響

# 確保 tokenizer 也正確
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# 二、開始推論，這裡假設任務是「繁體中文文章摘要」：
def summarize(text):
    prompt = f"請閱讀以下內文，並撰寫摘要：\n{text}\n\n摘要："

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 移除 prompt
    return result.split("摘要：")[-1].strip()

# 測試
text = "日本東京大學近日宣布，他們成功開發出一種能在低溫環境下穩定運作的量子晶片……"
print("摘要結果：", summarize(text))


* 進階 : 打包成API

如果你之後想要給網頁串接或當本地應用，可以用 Flask 快速包一個 API：

In [ ]:
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route("/summarize", methods=["POST"])
def summarize_api():
    data = request.get_json()
    text = data.get("text", "")
    summary = summarize(text)
    return jsonify({"summary": summary})

app.run(port=5000)
